# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902050


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:  10%|▉         | 3/31 [00:07<01:08,  2.44s/it]

Rendering models:  13%|█▎        | 4/31 [00:07<00:48,  1.80s/it]

Rendering models:  16%|█▌        | 5/31 [00:07<00:34,  1.33s/it]

Rendering models:  19%|█▉        | 6/31 [00:08<00:25,  1.02s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:13<00:00,  4.31it/s]

Rendering models:  94%|█████████▎| 29/31 [00:13<00:00,  4.27it/s]

Rendering models:  97%|█████████▋| 30/31 [00:13<00:00,  4.51it/s]

Rendering models: 100%|██████████| 31/31 [00:13<00:00,  4.22it/s]

equidad1                              0.141965
ElisabethB                            0.000149
Thedunkmasta                          0.000157
DrJackie                              0.000482
not-logged-in-305e3fb3bdeb23dda397    0.000568
Elemat                                0.003124
FrancescoGullo                        0.004246
EastonH                               0.000308
AllyOpps                              0.001703
not-logged-in-6cd9487f1f13f15d1751    0.000940
sn346808                              0.000292
jheadlee                              0.000305
not-logged-in-a2291ffe48f33f96a1a5    0.014951
Arbitrationer                         0.000697
ryanwong                              0.000161
koshmarus                             0.000233
Tenorclef                             0.000113
nbloewen                              0.000234
ccarles.100                           0.000999
ccarles.100                           0.000119
not-logged-in-92785a29f032fd3d6f9b    0.000466
VirK_QSO     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())